In [1]:
import torch
import glob
torch.cuda.empty_cache()

In [2]:
%run models/metric.py
%run models/inception.py
%run models/fid_score.py

In [3]:
PATH = "."

In [4]:
def get_sorted_scene_data_files(scene, data, file_name):
    files = glob.glob(os.path.join('data', scene, data, file_name))
    files.sort()
    return files

In [8]:
# Load a libtorch script file
def load_libtorch_script(train_id=None, model_script_name=None, checkpoint_name='model_best'):
    assert train_id is not None or model_script_name is not None
    
    if model_script_name is None:
        model_script_name = 'DNR-{}-{}_model.pt'.format(train_id, checkpoint_name)
    
    model_script_path = os.path.join(PATH, 'libtorch-models', model_script_name)
    #model_script_path = os.path.join(PATH, model_script_name)
        
    print(model_script_path)
    sm_loaded = torch.jit.load(model_script_path)
    sm_loaded.eval()
    return sm_loaded

In [6]:
_scene = 'scene0000_00'
model_script_names = ['DNR-0919_030156-model_best_model.pt',
                     'DNR-0922_171031-checkpoint-epoch{}-epoch-{}_model.pt'.format(100, 101),
                     'DNR-0930_130233-model_best-epoch-143_model.pt']
uv_size = (968, 1296)
batch_size = 5
device = 'cuda:0'

In [7]:
torch.cuda.empty_cache()
color_path = os.path.join('data', _scene, 'color')
uv_path = os.path.join('data', _scene, 'uv/scene0000_00_vh_clean_smartuv_75_0_0_no_aspect_enclosed')

fid = FIDScore(device=device)

print('Computing statistics of ground truth')
mg, sg = fid.compute_statistics_of_path(color_path, batch_size)
torch.cuda.empty_cache()

for name in model_script_names:
    print('Computing statistics of', name)
    with torch.no_grad():
        model = load_libtorch_script(model_script_name=name).to(device)
        model.eval()
        mp, sp = fid.compute_statistics_of_model(uv_path, model, uv_size, batch_size)
        
    score = fid.calculate_frechet_distance(mg, sg, mp, sp)
    torch.cuda.empty_cache()
    
    print(name, 'FID:', score)

  0%|          | 0/1116 [00:00<?, ?it/s]

Computing statistics of ground truth


100%|██████████| 1116/1116 [00:59<00:00, 18.75it/s]


Computing statistics of DNR-0919_030156-model_best_model.pt
./libtorch-models/DNR-0919_030156-model_best_model.pt


100%|██████████| 1116/1116 [08:50<00:00,  2.10it/s]


DNR-0919_030156-model_best_model.pt FID: 87.20296006202955
Computing statistics of DNR-0922_171031-checkpoint-epoch100-epoch-101_model.pt
./libtorch-models/DNR-0922_171031-checkpoint-epoch100-epoch-101_model.pt


100%|██████████| 1116/1116 [05:12<00:00,  3.57it/s]


DNR-0922_171031-checkpoint-epoch100-epoch-101_model.pt FID: 89.83337128484197
Computing statistics of DNR-0930_130233-model_best-epoch-143_model.pt
./libtorch-models/DNR-0930_130233-model_best-epoch-143_model.pt


100%|██████████| 1116/1116 [05:05<00:00,  3.66it/s]


DNR-0930_130233-model_best-epoch-143_model.pt FID: 92.43067008522667


In [10]:
torch.cuda.empty_cache()
model_script_names = ['DNR-0922_171031-checkpoint-epoch100-epoch-500_model.pt',
                     'DNR-0922_171031-checkpoint-epoch200-epoch-500_model.pt',
                      'DNR-0922_171031-checkpoint-epoch300-epoch-500_model.pt',
                      'DNR-0922_171031-checkpoint-epoch400-epoch-500_model.pt',
                      'DNR-0922_171031-checkpoint-epoch500-epoch-500_model.pt']
color_path = os.path.join('data', _scene, 'color')
uv_path = os.path.join('data', _scene, 'uv/scene0000_00_vh_clean_smartuv_75_0_0_no_aspect_enclosed')

fid = FIDScore(device=device)

print('Computing statistics of ground truth')
mg, sg = fid.compute_statistics_of_path(color_path, batch_size)
torch.cuda.empty_cache()

for name in model_script_names:
    print('Computing statistics of', name)
    with torch.no_grad():
        model = load_libtorch_script(model_script_name=name).to(device)
        model.eval()
        mp, sp = fid.compute_statistics_of_model(uv_path, model, uv_size, batch_size)
        
    score = fid.calculate_frechet_distance(mg, sg, mp, sp)
    torch.cuda.empty_cache()
    
    print(name, 'FID:', score)

  0%|          | 0/1116 [00:00<?, ?it/s]

Computing statistics of ground truth


100%|██████████| 1116/1116 [01:00<00:00, 18.39it/s]


Computing statistics of DNR-0922_171031-checkpoint-epoch100-epoch-500_model.pt
./libtorch-models/DNR-0922_171031-checkpoint-epoch100-epoch-500_model.pt


  0%|          | 0/1116 [00:00<?, ?it/s]

...

DNR-0922_171031-checkpoint-epoch300-epoch-500_model.pt FID: 81.12128569294146
Computing statistics of DNR-0922_171031-checkpoint-epoch400-epoch-500_model.pt
./libtorch-models/DNR-0922_171031-checkpoint-epoch400-epoch-500_model.pt


100%|██████████| 1116/1116 [04:59<00:00,  3.73it/s]


DNR-0922_171031-checkpoint-epoch400-epoch-500_model.pt FID: 81.12128574555766
Computing statistics of DNR-0922_171031-checkpoint-epoch500-epoch-500_model.pt
./libtorch-models/DNR-0922_171031-checkpoint-epoch500-epoch-500_model.pt


100%|██████████| 1116/1116 [04:59<00:00,  3.72it/s]


DNR-0922_171031-checkpoint-epoch500-epoch-500_model.pt FID: 81.12128617619186


In [7]:
torch.cuda.empty_cache()
model_script_names = glob.glob(os.path.join('libtorch-models', '*.pt'))
print(model_script_names)
color_path = os.path.join('data', _scene, 'color')
uv_path = os.path.join('data', _scene, 'uv/scene0000_00_vh_clean_smartuv_75_0_0_no_aspect_enclosed')

fid = FIDScore(device=device)

print('Computing statistics of ground truth')
mg, sg = fid.compute_statistics_of_path(color_path, batch_size)
torch.cuda.empty_cache()

for name in model_script_names:
    print('Computing statistics of', name)
    with torch.no_grad():
        model = load_libtorch_script(model_script_name=name).to(device)
        model.eval()
        mp, sp = fid.compute_statistics_of_model(uv_path, model, batch_size)
        
    score = fid.calculate_frechet_distance(mg, sg, mp, sp)
    torch.cuda.empty_cache()
    
    print(name, 'FID:', score)

['libtorch-models/DNR-0930_130233-model_best-epoch-76_model.pt', 'libtorch-models/DNR-0922_171031-checkpoint-epoch300-epoch-500_model.pt', 'libtorch-models/DNR-0918_151827-model_best_model.pt', 'libtorch-models/DNR-0930_115844-checkpoint-epoch100-epoch-101_model.pt', 'libtorch-models/DNR-0930_130233-checkpoint-epoch100-epoch-38_model.pt', 'libtorch-models/DNR-0922_014719-model_best_model.pt', 'libtorch-models/DNR-0820_000814-model_best_model.pt', 'libtorch-models/DNR-0921_090238-model_best_model.pt', 'libtorch-models/DNR-0917_170138-model_best_model.pt', 'libtorch-models/DNR-1006_140654-checkpoint-epoch150-epoch-151_model.pt', 'libtorch-models/DNR-0930_130233-checkpoint-epoch100-epoch-51_model.pt', 'libtorch-models/DNR-1006_144522-model_best-epoch-110_model.pt', 'libtorch-models/DNR-0922_171031-checkpoint-epoch400-epoch-500_model.pt', 'libtorch-models/DNR-0922_171031-checkpoint-epoch300-epoch-301_model.pt', 'libtorch-models/DNR-0930_130233-model_best-epoch-116_model.pt', 'libtorch-mode

  0%|          | 0/1116 [00:00<?, ?it/s]

Computing statistics of ground truth


100%|██████████| 1116/1116 [01:00<00:00, 18.47it/s]


Computing statistics of libtorch-models/DNR-0930_130233-model_best-epoch-76_model.pt
./libtorch-models/DNR-0930_130233-model_best-epoch-76_model.pt


...

libtorch-models/DNR-0918_101609-model_best_model.pt FID: 342.1333844093273
Computing statistics of libtorch-models/DNR-1006_140654-checkpoint-epoch50-epoch-51_model.pt
./libtorch-models/DNR-1006_140654-checkpoint-epoch50-epoch-51_model.pt


100%|██████████| 1116/1116 [04:59<00:00,  3.72it/s]


libtorch-models/DNR-1006_140654-checkpoint-epoch50-epoch-51_model.pt FID: 365.5040682736678
Computing statistics of libtorch-models/DNR-0922_171031-checkpoint-epoch200-epoch-500_model.pt
./libtorch-models/DNR-0922_171031-checkpoint-epoch200-epoch-500_model.pt


100%|██████████| 1116/1116 [05:00<00:00,  3.71it/s]


libtorch-models/DNR-0922_171031-checkpoint-epoch200-epoch-500_model.pt FID: 81.12128638449877


In [9]:
torch.cuda.empty_cache()
model_script_names = ['DNR-1006_225207-model_best-epoch-141_model.pt']
print(model_script_names)
color_path = os.path.join('data', _scene, 'color')
uv_path = os.path.join('data', _scene, 'uv/scene0000_00_vh_clean_smartuv_75_0_0_no_aspect_enclosed')

fid = FIDScore(device=device)

print('Computing statistics of ground truth')
mg, sg = fid.compute_statistics_of_path(color_path, batch_size)
torch.cuda.empty_cache()

for name in model_script_names:
    print('Computing statistics of', name)
    with torch.no_grad():
        model = load_libtorch_script(model_script_name=name).to(device)
        model.eval()
        mp, sp = fid.compute_statistics_of_model(uv_path, model, uv_size, batch_size)
        
    score = fid.calculate_frechet_distance(mg, sg, mp, sp)
    torch.cuda.empty_cache()
    
    print(name, 'FID:', score)

['DNR-1006_225207-model_best-epoch-141_model.pt']


  0%|          | 0/1116 [00:00<?, ?it/s]

Computing statistics of ground truth


  0%|          | 0/1116 [00:00<?, ?it/s]

Computing statistics of DNR-1006_225207-model_best-epoch-141_model.pt
./libtorch-models/DNR-1006_225207-model_best-epoch-141_model.pt


100%|██████████| 1116/1116 [04:58<00:00,  3.74it/s]


DNR-1006_225207-model_best-epoch-141_model.pt FID: 91.48227493715189
